<a href="https://colab.research.google.com/github/hebercamacho/ConsumidorGovBr-Extractor/blob/master/2_tratamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
%pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Ler planilha e preencher dataframe

In [111]:
#p/ ler excel e usar dataframes
import pandas as pd

#receber nome da planilha
planilha = input("Digite o nome da planilha. Não colocar a extensão (.xlsx): ")

#carregar planilha
try:
  df = pd.read_excel(planilha + '.xlsx', index_col=0, 
    names=['empresa', 'resolucao', 'data_reclamacao', 'cidade', 'estado', 'reclamacao', 'resposta', 'dias_depois_resposta', 'nota', 'avaliacao','segmento_empresa'])
except FileNotFoundError:
  print("Planilha não localizada!")

display(df)

Digite o nome da planilha. Não colocar a extensão (.xlsx): ReclamacoesConsumidor-4021-espólio_inventário_herança_inventariante


,empresa,resolucao,data_reclamacao,cidade,estado,reclamacao,resposta,dias_depois_resposta,nota,avaliacao,segmento_empresa
0,Claro TV,Não avaliado pelo consumidor,06/11/2022,Rio de Janeiro,RJ,A assinante original Maria de Fátima de Souza ...,"""Prezado(a) Sr(a). NELSON, em atenção à sua re...",3.0,NaN,<não há comentários do consumidor>,"Operadoras de Telecomunicações (Telefonia, Int..."
1,Vivo - Telefônica,Não Resolvido,21/07/2022,Vitória,ES,Na qualidade de inventariante do Espólio de Pe...,"Prezado Procon, Considerando o disposto no ...",6.0,1.0,Permaneço aguardando o retorno. Enviado o Term...,"Operadoras de Telecomunicações (Telefonia, Int..."
2,Oi Fixo,Não avaliado pelo consumidor,09/06/2022,Joinville,SC,"Eu, FABIANA VITORIO EDUVIGES, na qualidade de ...","Oi, sua solicitação foi atendida. Em anexo seg...",7.0,NaN,<não há comentários do consumidor>,"Operadoras de Telecomunicações (Telefonia, Int..."
3,Oi Fixo,Resolvido,26/04/2022,Caldas Novas,GO,"O titular da linha faleceu no ano passado, e e...","Em atenção à reclamação realizada, finalizamos...",7.0,5.0,"A empresa entrou em contato comigo, o atendent...","Operadoras de Telecomunicações (Telefonia, Int..."
4,Vivo - Telefônica,Não avaliado pelo consumidor,11/04/2022,Cáceres,MT,ESPOLOLIO DE ARNO RIEDER - após o falecimento ...,"Prezado (a) Cliente, Desta forma, segue Car...",8.0,NaN,<não há comentários do consumidor>,"Operadoras de Telecomunicações (Telefonia, Int..."
...,...,...,...,...,...,...,...,...,...,...,...
4016,Ágora Investimentos,Resolvido,23/11/2020,Niterói,RJ,"Sou o co-titular da conta do Bradesco,na qual ...",Prezado (a) Sr. (a) Odilon Informamos que a q...,8.0,1.0,"Informações imprecisas, demoradas e sempre na ...",Corretoras e Distribuidoras de Títulos e Inves...
4017,Rico Investimentos,Não Resolvido,02/10/2020,Taquara,RS,"Meu marido, também cliente da Rico veio a óbit...","Prezada Sra. Luciane, Informamos que a re...",14.0,2.0,"A empresa enviou TED , depois de ligarmos recl...",Corretoras e Distribuidoras de Títulos e Inves...
4018,Ágora Investimentos,Não Resolvido,06/09/2020,São Paulo,SP,"Como inventariante, estou realizando a transfe...","Prezado Sr. Rodrigo, Informamos que a quest...",15.0,1.0,<não há comentários do consumidor>,Corretoras e Distribuidoras de Títulos e Inves...
4019,modalmais,Não Resolvido,10/08/2020,Brasília,DF,Meu nome é Patrícia Inácio. Meu marido faleceu...,"Prezada Sra. Patricia, Em atendimento à dem...",10.0,1.0,Venho informar que a instituição enviou um bol...,Corretoras e Distribuidoras de Títulos e Inves...


#### Pré-tratamento

In [112]:
# remoção de duplicatas
start_len = len(df)
df.drop_duplicates(inplace=True)
end_len = len(df)
print(start_len-end_len,'linhas duplicadas removidas')
print(end_len,'linhas exclusivas restantes')

836 linhas duplicadas removidas
3185 linhas exclusivas restantes


In [113]:
# parseamento da coluna de data
from datetime import datetime
df['data_reclamacao'] = [datetime.strptime(data, '%d/%m/%Y') for data in df["data_reclamacao"]]
len_affected = len(df['data_reclamacao'])
print(len_affected,'datas reformatadas')

3185 datas reformatadas


In [114]:
# apagar tag "<não há comentários do consumidor>"
len_affected = len(df.loc[df['avaliacao'] == "<não há comentários do consumidor>"])
df.loc[df['avaliacao'] == "<não há comentários do consumidor>", 'avaliacao'] = ""
print(len_affected,'avaliações não realizadas. Tag <não há comentários do consumidor> removida.')

1920 avaliações não realizadas. Tag <não há comentários do consumidor> removida.


### Limpeza dos dados

#### Limpeza de dados sensíveis

In [115]:
import re

ContarPadroes = lambda row, pattern: len(re.findall(pattern, f"{row.reclamacao} {row.resposta} {row.avaliacao}"))

def ContarDadosSensiveis(df):
  data_count = 0
  email_count = 0
  valor_count = 0
  cnpj_count = 0
  cpf_count = 0
  tel_count = 0
  url_count = 0
  nr_count = 0

  for row in df.itertuples():
    #datas
    pattern = r"\b([\d]{1,2})[/-]([\d]{1,2})([/-][\d]{2,4})?\b"
    data_count += ContarPadroes(row, pattern)

    #emails
    pattern = r"\b[\w._%+-]+@[\w.-]+\.[\w]{2,}\b"
    email_count += ContarPadroes(row, pattern)

    #valores
    pattern = r"\bR?\$[ ]*[\d]+([\.,]?[\d]+)*([\.,]\d{2})?\b"
    valor_count += ContarPadroes(row, pattern)

    #CNPJs
    pattern = r"\b[\d]{2}[\.]?[\d]{3}[\.]?[\d]{3}[\/]?[\dX]{4,5}[-]?[\dX]{2}\b"
    cnpj_count += ContarPadroes(row, pattern)

    #CPFs
    pattern = r"\b([\dX]{3}[\.][\dX]{3}[\.][\dX]{3}[-][\dX]{2})\b"
    cpf_count += ContarPadroes(row, pattern)

    #Telefones
    pattern = r"((\b|\()([1-9]{2}\)?[ ]?)[\dx]{4,5}-?[\dX]{4})|(0800 ?\d{3} ?\d{4})\b"
    tel_count += ContarPadroes(row, pattern)

    #URLs
    pattern = r"\b(https?://)(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{2,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)\b"
    url_count += ContarPadroes(row, pattern)

    #Outros números
    pattern = r"\d+|[-Xx\.\d]{2,}"
    nr_count += ContarPadroes(row, pattern)

  #Relatório
  print("Dados sensíveis existentes:")
  print(data_count,"Datas")
  print(email_count,"E-mails")
  print(valor_count,"Valores monetários")
  print(cnpj_count,"CNPJs")
  print(cpf_count,"CPFs")
  print(tel_count,"Números de Telefones")
  print(url_count,"URLs")
  print(nr_count,"Números de contas, protocolos, CEPs, entre outros")


def LimparDadosSensiveis(texto: str):
  #is Nan
  if texto != texto or len(texto) < 2:
    return ""    

  #substituir datas pela label {data}
  pattern = r"\b([\d]{1,2})[/-]([\d]{1,2})([/-][\d]{2,4})?\b"
  texto = re.sub(pattern, " {data} ", texto)

  #substituir emails pela label {email}
  pattern = r"\b[\w._%+-]+@[\w.-]+\.[\w]{2,}\b"
  texto = re.sub(pattern, " {email} ", texto)

  #substituir valores monterarios pela label {valor}
  pattern = r"\bR?\$[ ]*[\d]+([\.,]?[\d]+)*([\.,]\d{2})?\b"
  texto = re.sub(pattern, " {valor} ", texto)

  #substituir CNPJs pela label {cnpj}
  pattern = r"\b[\d]{2}[\.]?[\d]{3}[\.]?[\d]{3}[\/]?[\dX]{4,5}[-]?[\dX]{2}\b"
  texto = re.sub(pattern, " {cnpj} ", texto)
  
  #substituir CPFs pela label {cpf}
  pattern = r"\b([\dX]{3}[\.][\dX]{3}[\.][\dX]{3}[-][\dX]{2})\b"
  texto = re.sub(pattern, " {cpf} ", texto)

  #substituir números de telefone pela label {telefone}
  pattern = r"((\b|\()([1-9]{2}\)?[ ]?)[\dx]{4,5}-?[\dX]{4})|(0800 ?\d{3} ?\d{4})\b"
  texto = re.sub(pattern, " {telefone} ", texto)

  #substituir URLs pela label {url}
  pattern = r"\b(https?://)(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{2,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)\b"
  texto = re.sub(pattern, " {url} ", texto)

  #substituir todos os demais números pela label {dígitos}
  pattern = r"\d+|[-Xx\.\d]{2,}"
  texto = re.sub(pattern, " {dígitos} ", texto)

  return texto

In [116]:
print('Antes da limpeza:')
ContarDadosSensiveis(df)

Antes da limpeza:
Dados sensíveis existentes:
4242 Datas
466 E-mails
1276 Valores monetários
33 CNPJs
1346 CPFs
1647 Números de Telefones
228 URLs
46318 Números de contas, protocolos, CEPs, entre outros


In [117]:
df['reclamacao'] = [LimparDadosSensiveis(reclamacao) for reclamacao in df['reclamacao']]
df['resposta'] = [LimparDadosSensiveis(resposta) for resposta in df['resposta']]
df['avaliacao'] = [LimparDadosSensiveis(avaliacao) for avaliacao in df['avaliacao']]

In [118]:
print('Depois da limpeza:')
ContarDadosSensiveis(df)

Depois da limpeza:
Dados sensíveis existentes:
0 Datas
0 E-mails
0 Valores monetários
0 CNPJs
0 CPFs
0 Números de Telefones
0 URLs
0 Números de contas, protocolos, CEPs, entre outros


#### Limpeza de gírias (internetês)

Aqui colocamos todos os termos que são abreviados pela linguagem informal da internet (internetês), e mapeamos para os termos corretos.
Isso é importante porque o método de correção ortográfica não corrige esse tipo de linguagem, e essas palavras podem prejudicar a contabilização dos termos.

In [119]:
import numpy as np

dicionario_internetes = [
    (r"\b(q)\b", "que"),
    (r"\b(oq)\b", "o que"),
    (r"\b(pq)\b", "porque"),
    (r"\b(vc)\b", "você"),
    (r"\b(vcs)\b", "vocês"),
    (r"\b(eh)\b", "é"),
    (r"\b(soh)\b", "só"),
    (r"\b(tava)\b", "estava"),
    (r"\b(d)\b", "de"),
    (r"\b(p\/?)\b", "para"),    
    (r"\b(p)\b", "para"),
    (r"\b(c\/?)\b", "com"),    
    (r"\b(c)\b", "com"),
    (r"\b(qdo?)\b", "quando"),
    (r"\b(qt)\b", "quanto"),
    (r"\b(qtd)\b", "quantidade"),
    (r"\b(vzs)\b", "vezes"),
    (r"\b(vz)\b", "vez"),
    (r"\b[nñ](aum)?\b", "não"),
    (r"\b(t[oô])\b", "estou"),
    (r"\bt[aá]h?\b", "está"),
    (r"\b(mto*)\b", "muito"),
    (r"\b(tbm?)\b", "também"),
    (r"\b(qro)\b", "quero"),
    (r"\b(ngm)\b", "ninguém"),
    (r"\b(qm)\b", "quem"),
    (r"\b(hj)\b", "hoje"),
    (r"\b(mnh)\b", "minha"),
    (r"\b(agr)\b", "agora")
]

def ContarGirias(df):
  df_girias = pd.DataFrame(columns=['contagem'], index=[giria for giria,_ in dicionario_internetes])

  for row in df.itertuples():
    for giria, correta in dicionario_internetes:
      #inicializacao
      if np.isnan(df_girias['contagem'][giria]):
        df_girias['contagem'][giria] = 0

      #realizando contagem
      df_girias['contagem'][giria] += ContarPadroes(row, giria)

  display(df_girias)

#função que substitui todas as girias de internetes pela palavra em norma culta
def SubstituirInternetes(texto):
  for giria, correta in dicionario_internetes:
    texto = re.sub(giria, correta, texto, flags=re.IGNORECASE)
  return texto

In [120]:
print('Antes da limpeza:')
ContarGirias(df)

Antes da limpeza:


,contagem
\b(q)\b,138
\b(oq)\b,1
\b(pq)\b,91
\b(vc)\b,7
\b(vcs)\b,46
\b(eh)\b,3
\b(soh)\b,1
\b(tava)\b,3
\b(d)\b,17
\b(p\/?)\b,47


In [121]:
df['reclamacao'] = [SubstituirInternetes(reclamacao) for reclamacao in df['reclamacao']]
df['resposta'] = [SubstituirInternetes(resposta) for resposta in df['resposta']]
df['avaliacao'] = [SubstituirInternetes(avaliacao) for avaliacao in df['avaliacao']]

In [122]:
print('Depois da limpeza:')
ContarGirias(df)

Depois da limpeza:


,contagem
\b(q)\b,0
\b(oq)\b,0
\b(pq)\b,0
\b(vc)\b,0
\b(vcs)\b,0
\b(eh)\b,0
\b(soh)\b,0
\b(tava)\b,0
\b(d)\b,0
\b(p\/?)\b,0


### Correção ortográfica

In [123]:
from spellchecker import SpellChecker
spell = SpellChecker(language='pt')

#recebe um texto com possiveis erros e faz a correção ortográfica
def CorrecaoOrtografica(texto: str):
  if len(texto) == 0:
    return texto
  texto_corrigido = spell.correction(texto)
  if texto_corrigido == texto or texto_corrigido is None:
    return texto 
  print('CORREÇÃO EFETUADA!')
  print('texto original: ',texto)
  print('texto corrigido:',texto_corrigido)
  return texto_corrigido

In [124]:
df['reclamacao'] = [CorrecaoOrtografica(reclamacao) for reclamacao in df['reclamacao']]
df['resposta'] = [CorrecaoOrtografica(resposta) for resposta in df['resposta']]
df['avaliacao'] = [CorrecaoOrtografica(avaliacao) for avaliacao in df['avaliacao']]

CORREÇÃO EFETUADA!
texto original:  Gratidão!
texto corrigido: gratidão
CORREÇÃO EFETUADA!
texto original:  Obrigada.
texto corrigido: obrigada
CORREÇÃO EFETUADA!
texto original:  satisfatório.
texto corrigido: satisfatório
CORREÇÃO EFETUADA!
texto original:  Obrigada.
texto corrigido: obrigada
CORREÇÃO EFETUADA!
texto original:  Excelente.
texto corrigido: excelente
CORREÇÃO EFETUADA!
texto original:  Obrigado.
texto corrigido: obrigado
CORREÇÃO EFETUADA!
texto original:  NEGATIVO.
texto corrigido: negativo


#### Definindo categoria e comprimento da reclamação

Cálculo das faixas de notas, para categorizar avaliações ruins, neutras e boas

In [125]:
notaMinima = 1.0
notaMaxima = 5.0
notaBoa = 4.0
notaRuim = 2.0
print(f'Ruim: {notaMinima} a {notaRuim}')
print(f'Neutra: {notaRuim+1} a {notaBoa-1}')
print(f'Boa: {notaBoa} a {notaMaxima}')

#definir categoria, sendo 0-50% Ruim, 50-80% neutro, 80-100% Boa
def defineCategoria(nota: float):
  return "Ruim" if nota <= notaRuim else "Boa" if nota >= notaBoa else "Neutra" if nota < notaBoa and nota > notaRuim else "Sem nota"

Ruim: 1.0 a 2.0
Neutra: 3.0 a 3.0
Boa: 4.0 a 5.0


Cálculo das categorias e comprimentos, complementando o mesmo DataFrame

In [126]:
#criar coluna com nome da categoria
df["categoria"] = [defineCategoria(float(nota)) for nota in df['nota']]
#criar coluna especificando comprimento da reclamacao
df['comprimento_reclamacao'] = [len(reclamacao) for reclamacao in df['reclamacao']]
#criar coluna especificando comprimento da resposta
df['comprimento_resposta'] = [len(resposta) if type(resposta) is str else 0 for resposta in df['resposta']]
#criar coluna especificando comprimento da avaliacao
df['comprimento_avaliacao'] = [len(avaliacao) if type(avaliacao) is str else 0 for avaliacao in df['avaliacao']]

In [128]:
display(df)

,empresa,resolucao,data_reclamacao,cidade,estado,reclamacao,resposta,dias_depois_resposta,nota,avaliacao,segmento_empresa,categoria,comprimento_reclamacao,comprimento_resposta,comprimento_avaliacao
0,Claro TV,Não avaliado pelo consumidor,2022-11-06,Rio de Janeiro,RJ,A assinante original Maria de Fátima de Souza ...,"""Prezado(a) Sr(a). NELSON, em atenção à sua re...",3.0,NaN,,"Operadoras de Telecomunicações (Telefonia, Int...",Sem nota,574,1146,0
1,Vivo - Telefônica,Não Resolvido,2022-07-21,Vitória,ES,Na qualidade de inventariante do Espólio de Pe...,"Prezado Procon, Considerando o disposto no ...",6.0,1.0,Permaneço aguardando o retorno. Enviado o Term...,"Operadoras de Telecomunicações (Telefonia, Int...",Ruim,299,962,75
2,Oi Fixo,Não avaliado pelo consumidor,2022-06-09,Joinville,SC,"Eu, FABIANA VITORIO EDUVIGES, na qualidade de ...","Oi, sua solicitação foi atendida. Em anexo seg...",7.0,NaN,,"Operadoras de Telecomunicações (Telefonia, Int...",Sem nota,1609,407,0
3,Oi Fixo,Resolvido,2022-04-26,Caldas Novas,GO,"O titular da linha faleceu no ano passado, e e...","Em atenção à reclamação realizada, finalizamos...",7.0,5.0,"A empresa entrou em contato comigo, o atendent...","Operadoras de Telecomunicações (Telefonia, Int...",Boa,675,452,97
4,Vivo - Telefônica,Não avaliado pelo consumidor,2022-04-11,Cáceres,MT,ESPOLOLIO DE ARNO RIEDER - após o falecimento ...,"Prezado (a) Cliente, Desta forma, segue Car...",8.0,NaN,,"Operadoras de Telecomunicações (Telefonia, Int...",Sem nota,464,770,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005,XP Investimentos,Não avaliado pelo consumidor,2020-07-26,Fortaleza,CE,Boa noite! Estou com um inventário finalizado...,Prezada Sra. Samantha. Esclarecemos que env...,8.0,NaN,,Corretoras e Distribuidoras de Títulos e Inves...,Sem nota,336,300,0
4006,Clear Corretora,Resolvido,2020-06-25,São Paulo,SP,"Boa tarde, O atendimento da clear é péssimo....","Olá Maria, Conforme o contato telefônico rea...",14.0,4.0,A empresa resolveu a contento e obtive a soluç...,Corretoras e Distribuidoras de Títulos e Inves...,Boa,725,754,137
4012,Ágora Investimentos,Não avaliado pelo consumidor,2021-10-06,Recife,PE,DESCUMPRIMENTO DE ALVARÁ JUDICIAL E DE {dígit...,Prezado (a) Sr. (a) Informamos que a tratat...,9.0,NaN,,Corretoras e Distribuidoras de Títulos e Inves...,Sem nota,465,586,0
4018,Ágora Investimentos,Não Resolvido,2020-09-06,São Paulo,SP,"Como inventariante, estou realizando a transfe...","Prezado Sr. Rodrigo, Informamos que a quest...",15.0,1.0,,Corretoras e Distribuidoras de Títulos e Inves...,Ruim,879,206,0


In [129]:
df.to_csv(f"{planilha}-tratada.csv")
df.to_excel(f"{planilha}-tratada.xlsx")